In [1]:
val pathToJar = System.getProperty("user.dir") + "/HistogramModel.jar"
val x = ammonite.ops.Path(pathToJar)
interp.load.cp(x)

pathToJar: String = "/Users/pauldanilin/Documents/HSE/CourseWorks/HistogramModel/Demonstration/HistogramModel.jar"
x: ammonite.ops.Path = /Users/pauldanilin/Documents/HSE/CourseWorks/HistogramModel/Demonstration/HistogramModel.jar

In [19]:
import ru.hse.se.ba.danilin.paul.histogram_model.histogram.Implicits._
import ru.hse.se.ba.danilin.paul.histogram_model.histogram.Histogram
import ru.hse.se.ba.danilin.paul.histogram_model.histogram.HistogramImpl

import $ivy.`org.plotly-scala::plotly-jupyter-scala:0.3.1`

import plotly._
import plotly.element.{Color => PColor}
import plotly.element._
import plotly.layout._
import plotly.JupyterScala._

import ru.hse.se.ba.danilin.paul.histogram_model.atomizers._
import ru.hse.se.ba.danilin.paul.histogram_model.queries._
import java.io.File
import javax.imageio.ImageIO.read

plotly.JupyterScala.init()

import ru.hse.se.ba.danilin.paul.histogram_model.histogram.Implicits._

import ru.hse.se.ba.danilin.paul.histogram_model.histogram.Histogram

import ru.hse.se.ba.danilin.paul.histogram_model.histogram.HistogramImpl


import $ivy.$                                             


import plotly._

import plotly.element.{Color => PColor}

import plotly.element._

import plotly.layout._

import plotly.JupyterScala._


import ru.hse.se.ba.danilin.paul.histogram_model.atomizers._

import ru.hse.se.ba.danilin.paul.histogram_model.queries._

import java.io.File

import javax.imageio.ImageIO.read



In [20]:
implicit class DrawableHistogram[E](hist: Histogram[E]) {
    def draw() = {
        val font = Font(10)
        val mapColors = Map(
            Red -> PColor.RGB(255,0,0),
            Green -> PColor.RGB(0, 153, 0),
            Yellow -> PColor.RGB(255, 255, 0),
            GreenYellow -> PColor.RGB(210, 255, 77),
            Blue -> PColor.RGB(0,0,255),
            LightBlue -> PColor.RGB(100,100,255))

        val xaxis = Axis("Элементарные элементы")
        val yaxis = Axis("Количество")
        val xs = hist.elementsPresent.toSeq.sortBy(_.toString)
        val marker = Marker(color = xs.map{case elem: Color => mapColors(elem)})
        val ys = xs.map(e => hist(e).toDouble)
        Bar(xs.map(_.toString), ys, marker = marker).plot()
    }
}

def readImages(folder: File) = {
    val files = (folder.isDirectory match {
      case true => folder.listFiles().toList
      case false => List(folder)
    }).filterNot(_.getName.startsWith("."))
    val names = files.map(_.getName.split('.')(0))
    val hists = files
      .filterNot(_.isHidden)
      .map(f => read(f))
      .map(_.toHistogram)
    names.zip(hists)
}

defined class DrawableHistogram
defined function readImages

In [5]:
val imagesHists = readImages(new File("images")).toMap.mapValues(_.normalize)

val imagesAliases = imagesHists.mapValues( hist => HistogramInput(hist))

for evaluated 0.45 seconds
map evaluated 3.77 seconds
for evaluated 0.00 seconds
map evaluated 0.02 seconds
for evaluated 0.00 seconds
map evaluated 0.01 seconds
for evaluated 0.01 seconds
map evaluated 0.01 seconds
for evaluated 0.02 seconds
map evaluated 0.02 seconds
for evaluated 0.00 seconds
map evaluated 0.00 seconds
for evaluated 0.12 seconds
map evaluated 0.23 seconds
for evaluated 0.00 seconds
map evaluated 0.01 seconds
for evaluated 0.00 seconds
map evaluated 0.01 seconds
for evaluated 0.00 seconds
map evaluated 0.01 seconds
for evaluated 0.00 seconds
map evaluated 0.01 seconds
for evaluated 0.00 seconds
map evaluated 0.01 seconds
for evaluated 0.00 seconds
map evaluated 0.01 seconds
for evaluated 0.00 seconds
map evaluated 0.01 seconds
for evaluated 0.40 seconds
map evaluated 0.91 seconds
for evaluated 0.00 seconds
map evaluated 0.01 seconds
for evaluated 0.00 seconds
map evaluated 0.01 seconds
for evaluated 0.00 seconds
map evaluated 0.01 seconds
for evaluated 0.00 seconds
m

imagesHists: Map[String, Histogram[Color]] = Map(
  "l1_53_50_54_40" -> HistogramImpl(Map(GREEN -> 0.49612403100775193, YELLOW -> 0.25193798449612403, GREEN YELLOW -> 0.25193798449612403)),
  "tiger" -> HistogramImpl(Map(RED -> 0.4363470923980478, GREEN -> 0.04863925883034163, BLUE -> 0.0013235172470841261, GREEN YELLOW -> 0.04723302175531475, YELLOW -> 0.4664571097692117)),
  "sea" -> HistogramImpl(Map(RED -> 0.4970628318999953, GREEN -> 0.002208750411203534, BLUE -> 7.51915033601203E-4, GREEN YELLOW -> 0.002161755721603459, LIGHT BLUE -> 7.51915033601203E-4, YELLOW -> 0.4970628318999953)),
  "l1_33_50_200_10" -> HistogramImpl(Map(RED -> 0.1004270313853137, GREEN -> 0.24293009227461404, BLUE -> 0.14760952011228237, GREEN YELLOW -> 0.24101890285782543, LIGHT BLUE -> 0.023591244363484337, YELLOW -> 0.24442320900648012)),
...
imagesAliases: Map[String, HistogramInput[Color]] = Map(
  "l1_53_50_54_40" -> HistogramInput(
    HistogramImpl(Map(GREEN -> 0.49612403100775193, YELLOW -> 0.25193

In [31]:
val pixelsAliases = Map(
                       "blue" -> SubhistogramInput(Set(Blue, LightBlue)),
                       "red" -> SubhistogramInput(Set(Red, Yellow)),
                       "green" -> SubhistogramInput(Set(GreenYellow, Green)),
                       "yellow" -> SubhistogramInput(Set(GreenYellow, Yellow)),
                       "cyan" -> SubhistogramInput(Set(Cyan)),
                       "magenta" -> SubhistogramInput(Set(Magenta)))

pixelsAliases: Map[String, SubhistogramInput[_27] forSome { type _27 >: Magenta.type with Cyan.type <: Color }] = Map(
  "magenta" -> SubhistogramInput(ru.hse.se.ba.danilin.paul.histogram_model.histogram.Implicits$SetUniverse@7f9a73aa),
  "blue" -> SubhistogramInput(ru.hse.se.ba.danilin.paul.histogram_model.histogram.Implicits$SetUniverse@3e42a8f9),
  "green" -> SubhistogramInput(ru.hse.se.ba.danilin.paul.histogram_model.histogram.Implicits$SetUniverse@7a0440d4),
  "yellow" -> SubhistogramInput(ru.hse.se.ba.danilin.paul.histogram_model.histogram.Implicits$SetUniverse@63927670),
  "cyan" -> SubhistogramInput(ru.hse.se.ba.danilin.paul.histogram_model.histogram.Implicits$SetUniverse@4f394e8b),
  "red" -> SubhistogramInput(ru.hse.se.ba.danilin.paul.histogram_model.histogram...

In [32]:
import ru.hse.se.ba.danilin.paul.histogram_model.queries.Input

implicit val aliases = (Query.standardAliases[Color] ++ imagesAliases ++ pixelsAliases).asInstanceOf[Map[String, Input[Color]]]

import ru.hse.se.ba.danilin.paul.histogram_model.queries.Input


aliases: Map[String, Input[Color]] = Map(
  "l1_53_50_54_40" -> HistogramInput(
    HistogramImpl(Map(GREEN -> 0.49612403100775193, YELLOW -> 0.25193798449612403, GREEN YELLOW -> 0.25193798449612403))
  ),
  "intersect" -> OperationInput(
    ru.hse.se.ba.danilin.paul.histogram_model.operations.Intersect$@5001c0b0
  ),
  "tiger" -> HistogramInput(
    HistogramImpl(Map(RED -> 0.4363470923980478, GREEN -> 0.04863925883034163, BLUE -> 0.0013235172470841261, GREEN YELLOW -> 0.04723302175531475, YELLOW -> 0.4664571097692117))
...

In [33]:
imagesHists("sea").draw

res32: String = "plot-1348209467"

In [23]:
val query = Query.fromString("blue")(aliases)
val result = query.execute(imagesHists("tiger")).left.get
result.draw

query: Query[Color] = ru.hse.se.ba.danilin.paul.histogram_model.queries.Query@232faf63
result: Histogram[Color] = HistogramImpl(Map(BLUE -> 0.0013235172470841261))
res22_2: String = "plot-1079623204"

In [15]:
result similar imagesHists("tiger")

res14: Double = 0.457711038961039

In [26]:
val q = Query.fromString("red | green")(aliases)

val sea = imagesHists("sea")

val res = q.execute(sea).left.get

res similar sea

q: Query[Color] = ru.hse.se.ba.danilin.paul.histogram_model.queries.Query@392ddae1
sea: Histogram[Color] = HistogramImpl(Map(RED -> 0.4970628318999953, GREEN -> 0.002208750411203534, BLUE -> 7.51915033601203E-4, GREEN YELLOW -> 0.002161755721603459, LIGHT BLUE -> 7.51915033601203E-4, YELLOW -> 0.4970628318999953))
res: Histogram[Color] = HistogramImpl(Map(RED -> 0.4970628318999953, YELLOW -> 0.4970628318999953, GREEN -> 0.002208750411203534))
res25_3: Double = 0.49927158231119884

In [27]:
val tiger = imagesHists("tiger")
tiger.draw

tiger: Histogram[Color] = HistogramImpl(Map(RED -> 0.4363470923980478, GREEN -> 0.04863925883034163, BLUE -> 0.0013235172470841261, GREEN YELLOW -> 0.04723302175531475, YELLOW -> 0.4664571097692117))
res26_1: String = "plot-1052034051"

In [28]:
val res = q.execute(tiger).left.get
res.draw
tiger similar sea

res: Histogram[Color] = HistogramImpl(Map(RED -> 0.4363470923980478, YELLOW -> 0.4664571097692117, GREEN -> 0.04863925883034163))
res27_1: String = "plot-370381557"
res27_2: Double = 0.9079266233336677

In [36]:
val query = Query.fromString("green|yellow")(aliases)
val result = query.execute(imagesHists("tiger")).left.get
result.draw

query: Query[Color] = ru.hse.se.ba.danilin.paul.histogram_model.queries.Query@4ad0e5ba
result: Histogram[Color] = HistogramImpl(Map(GREEN -> 0.04863925883034163, GREEN YELLOW -> 0.04723302175531475, YELLOW -> 0.4664571097692117))
res35_2: String = "plot-486522627"

In [37]:
val newQuery = new Query(query.injectHistogram(imagesHists("sea")))
val result = newQuery.execute(imagesHists("tiger")).left.get
result.draw

newQuery: Query[Color] = ru.hse.se.ba.danilin.paul.histogram_model.queries.Query@452191fc
result: Histogram[Color] = HistogramImpl(Map(GREEN -> 0.002208750411203534, GREEN YELLOW -> 0.002161755721603459, YELLOW -> 0.4970628318999953))
res36_2: String = "plot-781808994"

In [38]:
val result = imagesHists("sea") + imagesHists("tiger")
result.draw

result: Histogram[Color] = HistogramImpl(Map(RED -> 0.4970628318999953, GREEN -> 0.04863925883034163, BLUE -> 0.0013235172470841261, GREEN YELLOW -> 0.04723302175531475, LIGHT BLUE -> 7.51915033601203E-4, YELLOW -> 0.4970628318999953))
res37_1: String = "plot-818954520"

In [17]:
result(Blue)

res16: Double = 0.0012987012987012987

In [18]:
imagesHists("sea") similar imagesHists("tiger")

res17: Double = 0.9235000254056485

In [24]:
import ru.hse.se.ba.danilin.paul.histogram_model.histogram.Implicits._
import org.json4s.native.Serialization.{write, read => readJson}
import java.io.PrintWriter

val json = write(result)
val writer = new PrintWriter(new File("test result.json"))
writer.write(json)
writer.close()

import ru.hse.se.ba.danilin.paul.histogram_model.histogram.Implicits._

import org.json4s.native.Serialization.{write, read => readJson}

import java.io.PrintWriter


json: String = """
{"histogram":{"GREEN":0.04772727272727273,"BLUE":0.0012987012987012987,"CYAN":0.002922077922077922,"YELLOW":0.4811882989854875,"RED":0.4811882989854875,"MAGENTA":0.062175324675324675}}
"""
writer: PrintWriter = java.io.PrintWriter@464ed468

In [23]:
val writer = new PrintWriter(new File("test result.txt"))
writer.write(imagesHists("sea") similar imagesHists("tiger") toString)
writer.close()

writer: PrintWriter = java.io.PrintWriter@4997974